In [74]:
# pip als Paketmanager
#! pip install -q pyscipopt
#! pip install pandas
#!pip install openpyxl

In [75]:
from pyscipopt import Model, quicksum

**Optimierungsmodell für den Kauf und Verkauf von Strom auf dem Strommarkt**

In [76]:
# Erstellen einer Modellinstanz
scip = Model()

**Indexmenge**

In [77]:
# Investition über Zeitraum j Tage
j = 1

H = [n for n in range(1, 24*j+1)]
R = [r for r in range(0, 25)]
# Erklärung:
# es gibt 26 Risikoabstufungen, höchstes R -> höchstes Risiko, r=0, kein Risiko
#-> kein Risiko + normaler EW, Risikoklasse 1=4% Risiko und EW ergibt sich aus top 96% der Preisprognosen für h, etc.

# Risikowahrscheinlichkeiten
prob_r = {}
for r in R:
    prob_r[r] = 1 - r/25



**Batterie-Systemspezifikationen**

In [78]:
wirkungsgrad_wechselrichter = 0.985

wirkungsgrad_laden = 0.975

round_trip_efficiency = 0.95
entlade_verlust = wirkungsgrad_laden - round_trip_efficiency

wirkungsgrad_systemeingang = wirkungsgrad_wechselrichter * wirkungsgrad_laden
wirkungsgrad_systemausgang = (1-(entlade_verlust / wirkungsgrad_laden)) * wirkungsgrad_wechselrichter



f_e = wirkungsgrad_systemeingang # Faktor Einkauf
f_v = wirkungsgrad_systemausgang # Faktor Verkauf

# Durch investition verbesserte Wirkungsgrad-Faktoren: (Effizienzverluste halbiert), Kosten:
f_e_pro = 1-((1 - f_e)*0.5)
f_v_pro = 1-((1 - f_v)*0.5)


nennkapazität = 40 # MWh brutto
nennkapazität_pro = 50 # Kosten:

lademinimum = 0.2 # 20%
lademaximum = 1 # 100%
anfangsbestand = 0.5 # 50%

nettokapazität = zyklus = nennkapazität * (lademaximum - lademinimum) # MWh netto
nettokapazität_pro = nennkapazität_pro * (lademaximum - lademinimum) # Kosten:

zykluskosten = 1500 # € / zyklus
zykluskosten_pro = 1000
mwh_zykluskosten = zykluskosten / zyklus # € / MWh

erlaubte_zyklen_pro_tag = 2
erlaubte_zyklen_pro_tag_pro = 3

a = anfangsbestand * nennkapazität # MWh Anfangs- und Endbestand
u = lademinimum * nennkapazität # MWh Untergrenze Batteriekapazität
o = lademaximum * nennkapazität # MWh Obergrenze Batteriekapazität

c = 0.5 # nennkapazität / h
c_pro = 1 # Kosten:

# Prozentuale penalty, wenn nicht Zuschlag
penalty_anteil = 1
penalty = 1 + penalty_anteil



# -> Wir haben vier Investitionsentscheidungen (f_e_pro und f_v_pro, c_pro, nennkapazität_pro und erlaubte_zyklen_pro_tag_pro)

**Vorhersagedaten**

In [79]:
import pandas as pd

prognose = pd.read_excel('Preisprognosen.xlsx')

# Erwartungswerte nach Risikoklasse und Stunde für Einkauf

preis_verkauf_h_r = {}
preis_einkauf_h_r = {}

gebotspreis_verkauf_h_r = {}
gebotspreis_einkauf_h_r = {}

import math
for h in range(1,25):
  # Berechnung von "Standard-EW" für Stunde h
  stundenprognose = prognose[prognose['Stunde'] == h]
  #Sortiere stundenprognose nach dem Wert in Spalte "Strompreis"
  stundenprognose = stundenprognose.sort_values(by='Strompreis')
  # Index zurücksetzen
  stundenprognose = stundenprognose.reset_index(drop=True)
  #print(stundenprognose)
  for r in R:
    #iteriere außerdem j*1 über den df
    for i in range(0, j):
    # Nehme den Durschnitt von der Spalte "Strompreis" vom Index r bis zum höchsten Index
      preis_verkauf_h_r[(h+i*24, r)] = stundenprognose.loc[r:24, 'Strompreis'].mean()
      preis_einkauf_h_r[(h+i*24, r)] = stundenprognose.loc[0:24-r, 'Strompreis'].mean()
      gebotspreis_verkauf_h_r[(h+i*24, r)] = stundenprognose.loc[r:24, 'Strompreis'].min()
      gebotspreis_einkauf_h_r[(h+i*24, r)] = stundenprognose.loc[0:24-r, 'Strompreis'].max()

# Für die Berechnung der Penalty möchten wir den Durchschnittspreis der umliegenden Stunden zu jeder Stunde haben
moving_average_h = {}
for h in range(1,25):
  stundenprognose = prognose[prognose['Stunde'].isin([h-1,h,h+1])]
  for i in range(0, j):
    moving_average_h[h+i*24] = stundenprognose['Strompreis'].mean()


**Entscheidungsvariablen**

In [80]:
# Für jede Risikoklasse und jede Stunde eine Mengen-Variable
e_h_r={}
v_h_r={}

# Investitionsentscheidungen
inv_c_rate = scip.addVar(vtype="B", name=f"Investition_c-Rate")
inv_nennkapazität = scip.addVar(vtype="B", name=f"Investition_Nennkapa")
inv_zyklen_pro_tag = scip.addVar(vtype="B", name=f"Investition_Zyklen_pro_Tag")
inv_wirkungsgrad = scip.addVar(vtype="B", name=f"Investition_Wirkungsgrad")

for h in H:
    #  Für jede Risikoklasse und jede Stunde eine Mengen-Variable, ob man zu dieser Stunde bei dieser Risikoklasse die entsprechende Menge einkaufen möchte
    for r in R:
      e_h_r[(h, r)] = scip.addVar(vtype='C', lb=0, ub=None, name=f"e_{h}_{r}")
      v_h_r[(h, r)] = scip.addVar(vtype='C', lb=0, ub=None, name=f"v_{h}_{r}")


# **Zielfunktion**

In [81]:
gewinn_kauf_verkauf = quicksum(((preis_verkauf_h_r[(h, r)] * v_h_r[(h, r)] * prob_r[r] +\
                                 moving_average_h[h] * v_h_r[(h, r)] * (1-prob_r[r]) * 1/penalty) -\

                                (preis_einkauf_h_r[(h, r)] * e_h_r[(h, r)] * prob_r[r] +\
                                 moving_average_h[h] * e_h_r[(h, r)] * (1-prob_r[r]) * penalty ))\
                               
                               for r in R for h in H )


# Bei 3 Zyklen Investition kostet der dritte Zyklus nur 1000 mehr pro Tag (anstatt 1.500), also Skaleneffekte. Quelle: haben wir uns ausgedacht
zykluskosten_sum = (zykluskosten * erlaubte_zyklen_pro_tag + (zykluskosten_pro * (erlaubte_zyklen_pro_tag_pro - erlaubte_zyklen_pro_tag) * inv_zyklen_pro_tag)) * j

# Investitionskosten, direkt mit der Binärvariable multipliziert, sodass sie nur Anwendung finden wenn man sich für die Inv entscheidet
investitionskosten_c_rate = 40_000 * inv_c_rate *(j/365)
investitionskosten_nennkapazität = 40_000 *  inv_nennkapazität *(j/365)
investitionskosten_zyklen_pro_tag = 40_000 * inv_zyklen_pro_tag *(j/365)
investitionskosten_wirkungsgrad = 100_000 * inv_wirkungsgrad *(j/365)


scip.setObjective(gewinn_kauf_verkauf - zykluskosten_sum - (investitionskosten_nennkapazität + investitionskosten_c_rate + inv_zyklen_pro_tag + inv_wirkungsgrad), sense="maximize")

***Nebenbedingungen/ Restriktionen***

In [82]:
f_e_inv = ((f_e*(1-inv_wirkungsgrad))+(f_e_pro*inv_wirkungsgrad))
f_v_inv = ((f_v*(1-inv_wirkungsgrad))+(f_v_pro*inv_wirkungsgrad))
c_inv = ((c*(1-inv_c_rate))+(c_pro*inv_c_rate)) 

erlaubte_zyklen_pro_tag_inv = ((erlaubte_zyklen_pro_tag*(1-inv_zyklen_pro_tag))+(erlaubte_zyklen_pro_tag_pro*inv_zyklen_pro_tag))

nettokapazität_inv = ((nettokapazität*(1-inv_nennkapazität))+nettokapazität_pro*inv_nennkapazität)

# Ladestand zur Stunde 0 = Ladestand zur Stunde 24, also Summe Lademenge und Entlademenge gleich
scip.addCons(quicksum(((e_h_r[(h,r)] * f_e_inv) - (v_h_r[(h,r)] / f_v_inv)) for r in R for h in H) == 0, name="Anfangs- und Endbestand gleich")
print("added: Anfangs- und Endbestand gleich")

# Maximale Ladezyklen am pro Tag anhand der Einkaufsmenge (mit Faktor = Lademenge), alternativ anhand der Verkaufsmenge
scip.addCons(quicksum((e_h_r[(h,r)] * f_e_inv) for r in R for h in H) <= (erlaubte_zyklen_pro_tag_inv * nettokapazität_inv * 1 * j), name="Maximale Ladezyklen pro Tag")
print("added: Maximale Ladezyklen pro Tag")

# Mindestladestand nicht unterschritten und Höchstladestand nicht überschritten
for h in H:
    if h % 24 == 0: print(f"Stunde {h} wird hinzugefügt")
    H_t =  [n for n in range(1, h+1)]
    scip.addCons( (a + quicksum(((e_h_r[(t, r)] * f_e_inv) - (v_h_r[(t, r)] / f_v_inv)) for r in R for t in H_t)) >= u, name=f"Mindestladestand zum Zeitpunkt t={h}")
    scip.addCons( (a + quicksum(((e_h_r[(t, r)] * f_e_inv) - (v_h_r[(t, r)] / f_v_inv)) for r in R for t in H_t)) <= o, name=f"Maximalladestand zum Zeitpunkt t={h}")

# Lade- und Entladeleistung begrenzt (C-Rate)
for h in H:
    if h % 24 == 0: print(f"Stunde {h} wird hinzugefügt")
    scip.addCons(quicksum(((e_h_r[(h, r)] * f_e_inv) + (v_h_r[(h, r)] / f_v_inv)) for r in R) <= c_inv * nettokapazität_inv, name=f"Lade-/Entladeleistung der Stunde h={h}")
    
# Investitionsentscheidungen
# scip.addCons(inv_c_rate == 0, name="Test keine Investition")
# scip.addCons(inv_nennkapazität == 0, name="Test keine Investition")
# scip.addCons(inv_zyklen_pro_tag == 0, name="Test keine Investition")
# scip.addCons(inv_wirkungsgrad == 0, name="Test keine Investition")

added: Anfangs- und Endbestand gleich
added: Maximale Ladezyklen pro Tag
Stunde 24 wird hinzugefügt
Stunde 24 wird hinzugefügt


**Berechnung der Lösung**

In [83]:
scip.setIntParam("display/verblevel", 5)  # Set verbosity level to 5


scip.optimize()
# Status des Solvers
status = scip.getStatus()
print(f"Status des Solvers: {status} \n")

if status == "optimal":
    print('LÖSUNG:')
    print('Zielfunktionswert (Gewinn) =', scip.getObjVal())
    for h in H:
      v_h = 0
      e_h = 0
      risikoklasse_v = None
      risikodict_v = {}
      risikoklasse_e = None
      risikodict_e = {}
      risikodict_v[r] = scip.getVal(v_h_r[(h, r)])
      risikodict_e[r] = scip.getVal(e_h_r[(h, r)])
      for r in R:
        risikodict_v[r] = scip.getVal(v_h_r[(h, r)])
        risikodict_e[r] = scip.getVal(e_h_r[(h, r)])
        if scip.getVal(v_h_r[(h, r)]) > 0:
          risikoklasse_v = r

        if scip.getVal(e_h_r[(h, r)]) > 0:
          risikoklasse_e = r

        v_h += scip.getVal(v_h_r[(h, r)])
        e_h += scip.getVal(e_h_r[(h, r)])  
      print("EINKAUF Stunde", h, " : " , e_h)
      if risikoklasse_e != None: print("EINKAUF Risikoklasse Stunde", h, " : ", risikoklasse_e)
      #print("EINKAUF Risiko: ", risikodict_e)
      print("Verkauf Stunde", h, " : " , v_h)
      if risikoklasse_v != None: print("VERKAUF Risikoklasse Stunde", h, " : ", risikoklasse_v)
      #print("VERKAUF Risiko: ", risikodict_v)
else:
    print('Problem hat keine Lösung')

LP Solver <Soplex 7.1.1>: barrier convergence tolerance cannot be set -- tolerance of SCIP and LP solver may differ
LP Solver <Soplex 7.1.1>: fastmip setting not available -- SCIP parameter has no effect
Status des Solvers: optimal 

LÖSUNG:
Zielfunktionswert (Gewinn) = 1989.195901821765
EINKAUF Stunde 1  :  0.0
Verkauf Stunde 1  :  0.0
EINKAUF Stunde 2  :  0.0
Verkauf Stunde 2  :  0.0
EINKAUF Stunde 3  :  0.0
Verkauf Stunde 3  :  0.0
EINKAUF Stunde 4  :  20.404259389147487
EINKAUF Risikoklasse Stunde 4  :  0
Verkauf Stunde 4  :  0.0
EINKAUF Stunde 5  :  0.0
Verkauf Stunde 5  :  0.0
EINKAUF Stunde 6  :  0.0
Verkauf Stunde 6  :  0.0
EINKAUF Stunde 7  :  0.0
Verkauf Stunde 7  :  0.0
EINKAUF Stunde 8  :  0.0
Verkauf Stunde 8  :  31.355897435897447
VERKAUF Risikoklasse Stunde 8  :  3
EINKAUF Stunde 9  :  0.0
Verkauf Stunde 9  :  0.0
EINKAUF Stunde 10  :  0.0
Verkauf Stunde 10  :  0.0
EINKAUF Stunde 11  :  0.0
Verkauf Stunde 11  :  0.0
EINKAUF Stunde 12  :  0.0
Verkauf Stunde 12  :  0.0
EIN

In [84]:
print(f"C-Rate Investment = {"Ja" if scip.getVal(inv_c_rate) else "Nein"} von {c} auf {c_pro}")
print(f"Nennkapazität erhöhen: {"Ja" if scip.getVal(inv_nennkapazität) else "Nein"} von {nennkapazität} auf {nennkapazität_pro} MWh")
print(f"Zyklen pro Tag erhöhen: {"Ja" if scip.getVal(inv_zyklen_pro_tag) else "Nein"} von {erlaubte_zyklen_pro_tag} auf {erlaubte_zyklen_pro_tag_pro}")
print(f"Wirkungsgrad erhöhen: {"Ja" if scip.getVal(inv_wirkungsgrad) else "Nein"} von {f_e} auf {f_e_pro} und {f_v} auf {f_v_pro}")


C-Rate Investment = Ja von 0.5 auf 1
Nennkapazität erhöhen: Ja von 40 auf 50 MWh
Zyklen pro Tag erhöhen: Nein von 2 auf 3
Wirkungsgrad erhöhen: Ja von 0.960375 auf 0.9801875 und 0.9597435897435898 auf 0.9798717948717949


In [85]:
sum_v = 0
sum_e = 0
for h in H: 
    sum_v += scip.getVal(v_h_r[(h, r)])
    sum_e += scip.getVal(e_h_r[(h, r)])
    
print(sum_v)
print(sum_e)

0.0
0.0


In [86]:
# Ergebnisse in Excel speichern

import pandas as pd
from openpyxl import load_workbook


# File and worksheet details
file_path = "Ergebnisse.xlsx"

sheet_name = "var3"


# Prepare the solution data for export
solution_data = {
    "Hour": [h for h in H],
    "Einkauf": [],
    "Verkauf": [],
    "Risikoklasse": [],
    "Gebotspreis": [],
}

# Populate the solution_data dictionary
for h in H:
    v_h = 0
    e_h = 0
    risikoklasse_v = None
    risikoklasse_e = None
    for r in R:
        # Update the Verkauf and Einkauf values for the current hour
        v_h += scip.getVal(v_h_r[(h, r)])
        e_h += scip.getVal(e_h_r[(h, r)])
        # Determine the risk class for Verkauf and Einkauf
        if scip.getVal(v_h_r[(h, r)]) > 0:
            risikoklasse_v = r
        if scip.getVal(e_h_r[(h, r)]) > 0:
            risikoklasse_e = r

    # Append rounded values and risk classes to the solution data
    solution_data["Einkauf"].append(round(e_h, 3))
    solution_data["Verkauf"].append(round(v_h, 3))
    solution_data["Risikoklasse"].append(
        risikoklasse_v if round(v_h, 3) > 0 else risikoklasse_e if round(e_h, 3) > 0 else ""
    )
    solution_data["Gebotspreis"].append(
        gebotspreis_verkauf_h_r[(h, risikoklasse_v)] if round(v_h, 3) > 0 else gebotspreis_einkauf_h_r[(h, risikoklasse_e)] if round(e_h, 3) > 0 else ""
    )

# Convert to a DataFrame
solution_df = pd.DataFrame(solution_data)

# Write to Excel
try:
    # Load the workbook to check for existing worksheets
    workbook = load_workbook(file_path)
    if sheet_name not in workbook.sheetnames:
        # If the worksheet doesn't exist, create it
        print(f"Worksheet '{sheet_name}' not found. Creating it...")
        with pd.ExcelWriter(file_path, mode="a", engine="openpyxl") as writer:
            solution_df.to_excel(writer, sheet_name=sheet_name, index=False)
    else:
        # If the worksheet exists, overwrite it
        print(f"Worksheet '{sheet_name}' found. Writing solution...")
        with pd.ExcelWriter(file_path, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
            solution_df.to_excel(writer, sheet_name=sheet_name, index=False)
except FileNotFoundError:
    # If the file doesn't exist, create it and write the solution
    print(f"File '{file_path}' not found. Creating it and writing solution...")
    with pd.ExcelWriter(file_path, mode="w", engine="openpyxl") as writer:
        solution_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Solution written to '{file_path}' in the '{sheet_name}' worksheet.")


Worksheet 'var3' not found. Creating it...
Solution written to 'Ergebnisse.xlsx' in the 'var3' worksheet.
